In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
import math

还是使用垃圾邮件的数据集来测试

In [2]:
df = pd.read_table('./trec06c-utf8/label/index', sep=' ', header=None, names=['target','path'])
df['path'] = df['path'].str.replace('./data','/trec06c-utf8/data_cut')
emails = []

for i,j in df.iterrows():
    with open(j['path'], encoding='utf-8') as f:
        email = f.read()
        emails.append(''.join(email.split("\n\n",1)[1:]))

In [3]:
#emails = np.array(emails)
train_x, test_x, train_y, test_y = train_test_split(emails, df['target'].values, test_size=0.2, random_state=8)
print(type(train_x), type(test_x))

<class 'list'> <class 'list'>


In [5]:
vectorizer = CountVectorizer()
train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)
print(train_x.shape, test_x.shape)
print(type(train_x), type(test_x))
np.array(train_x)
np.array(test_y)
print(train_x.shape, test_x.shape)
print(type(train_x), type(test_x))

(51696, 181154) (12924, 181154)
<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'>
(51696, 181154) (12924, 181154)
<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'>



In [9]:
train_y[train_y == 'spam'] = 1
train_y[train_y == 'ham' ] = 0
print(len(train_y[train_y == 1]))
print(len(train_y[train_y == 0]))
test_y[test_y == 'spam'] = 1
test_y[test_y == 'ham' ] = 0

34357
17339


In [10]:
class NavieBayes(object):
    def __init__(self):
        self.PT = 0
        self.PF = 0
        self.lPT = 0
        self.lPF = 0
        self.PfT = []
        self.PfF = []
        
    def fit(self, X, Y):
        doc_num = len(Y)
        self.PT = len(Y[Y == 1])/doc_num # problity of T label
        self.PF = len(Y[Y == 0])/doc_num # problity of F label
        self.lPT = np.log((len(Y[Y == 1])+1)/doc_num) # log of problity of T label
        self.lPF = np.log((len(Y[Y == 0])+1)/doc_num) # log of problity of F label
        X[X>0] = 1
        feature_num = X.sum(axis=0)
        T = np.where(Y==1)
        F = np.where(Y==0)
        self.PfT = np.log((X[T].sum(axis=0)+1)/len(Y[Y==1]))
        self.PfF = np.log((X[F].sum(axis=0)+1)/len(Y[Y==0]))

    def predict(self, X, Y):
        P_Y = []
        PT = self.lPT + (X*self.PfT.T)
        PF = self.lPT + (X*self.PfF.T)
        for i in range(len(PT)):
            P_Y.append(int(1)) if PT[i] > PF[i] else P_Y.append(int(0))
        return P_Y

In [11]:
NB = NavieBayes()
NB.fit(train_x, train_y)
predict_y = NB.predict(test_x, test_y)
predict_y = np.array(predict_y)
#test_acc = accuracy_score(test_y, predict_y)
def acc(y_target, y_predict):
    return (y_target==y_predict).mean()
test_acc = acc(test_y, predict_y)
print("Accuracy by BernoulliNB is : %.6f" %(test_acc))

Accuracy by BernoulliNB is : 0.970520
